# Using an API without an access key (PubMed)

## Parameter strings

Most APIs allow the client to specify characteristics of the data to be retrieved through parameters attached to the endoint URL as a query string. 

The *query string* is separated from the endpoint URL by the `?` character. 

Each *key/value pair* is separated by the `&` character.

*Keys and values* are separated by the `=` character.

Example:
- endpoint URL: `https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi`
- first key: `db`
- first value: `pubmed`
- second key/value pair: `retmode=json`

GET URL:
```
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json
```

If a value contains spaces or any other character that is illegal in a URL, it must be *URL encoded*.  Spaces can be replaced by the `+` character.  Otherwise, the unicode equivalents are specified.  You can manually URL encode a string using a URL encoder website like [this one](https://meyerweb.com/eric/tools/dencoder/).  
However, the `requests` module will automatically encode the URL for you, so generally you don't have to worry about that.

## Generating parameter strings in Postman

Open Postman and paste the endpoint URL from above into the GET text box. Click on the `Params` tab and add the key value pairs listed above, plus a value of `5` for the key `retmax`.  Create a key for `term`.

![Postman parameters](https://raw.githubusercontent.com/HeardLibrary/digital-scholarship/master/code/scrape/pylesson/images/postman-params.png)

Go to the [URL encoder website](https://meyerweb.com/eric/tools/dencoder/) and paste in the search string `(biological sciences[ad] AND vanderbilt[ad])`. 

![URL encoder](https://raw.githubusercontent.com/HeardLibrary/digital-scholarship/master/code/scrape/pylesson/images/url-encoder.png)

Click `Encode`, then copy the encoded string.  Paste it into Postman as the valut for the `term` key.

![Postman constructed URL](https://raw.githubusercontent.com/HeardLibrary/digital-scholarship/master/code/scrape/pylesson/images/postman-url.png)

Clicking on the URL in the GET text box will show you how Postman has constructed the entire query string for you.  Click the `Send` button and examine the JSON in the response body.

## Generating parameter strings in Python

The `requests` module will automatically construct the parameters string for you if you pass the key/value pairs into the `.get()` method as a dictionary value for the `params` key. The following code generates the URL shown in the first example:

```
endpointUrl = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
paramDict = {
    'db': 'pubmed',
    'retmode': 'json'
}
response = requests.get(endpointUrl, params=paramDict)
```

## Discovering appropriate keys and values for parameter strings

To know what parameters are required to carry out the query you want, you need to look at the API developer guide.  The section of the NCBI developer guide for searching PubMed is [here](https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.ESearch).  To do a search of PubMed, the `db` and `term` (search term) parameters are required.  

The API defaults to returning XML results, so if we want to get JSON instead, we need to include `json` as a value for `retmode` (the API doesn't seem to process `Accept` headers).  We can also specify the number of results to return using `retmax` if we don't want the default 20 results.

The following script queries the NCBI Entrez database for articles that include "Biological Sciences" and "Vanderbilt", and requests 5 results.  

In [ ]:
import requests
import json
import webbrowser
import xml.etree.ElementTree as et
from time import sleep

# tool name and email address should be sent with all requests
# see https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.ESearch
emailAddress = '' # put your email address here
toolName = 'testApiScript' # give your application a name here
endpointUrl = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
searchString = '(biological sciences[ad] AND vanderbilt[ad])'
limit = 5

paramDict = {
    'tool': toolName, 
    'email': emailAddress,
    'db': 'pubmed',
    'retmode': 'json',
    'retmax': limit, 
    'term': searchString
}
response = requests.get(endpointUrl, params=paramDict)

# show the full URL with parameters and URL encoding
print(response.url)
print()

data = response.json()
prettifiedJson = json.dumps(data, indent=4, sort_keys=True)
print('prettified JSON: ')
print(prettifiedJson)

In order to do something with the results, we need to extract the retrieved article identifiers, called "UID"s, from the JSON. After running the query, you can see that the JSON isn't too complicated. The first level dictionary has only two items, with `esearchresult` as the key containing the part we want.  The value of `esearchresult` is another dictionary, with the key `idlist` containing the list of UIDs that we want. The expression that pulls that dictionary from the data is:

```
data['esearchresult']['idlist']
```

The following code opens the first of the UIDs in a browser tab by hacking the form of the PubMed URLs for articles:

In [ ]:
uidList = data['esearchresult']['idlist']
articleUrl = 'https://www.ncbi.nlm.nih.gov/pubmed/?term=' + uidList[0]
print(articleUrl)
success = webbrowser.open_new_tab(articleUrl)

## Additional API calls to retrieve more data

If we want to fetch metadata about the articles, we need to make calls to another endpoint: `https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi`.  The Developer Guide for that endpoint is [here](https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch).  Unfortunately, that endpoint doesn't support JSON, so the results must be retrieved as XML and processed into a form that Python can read.  The `xml.etree.ElementTree` module is useful for that purpose.

The code below loops through the UIDs one at a time and fetches XML data about the publication. It extracts the article title, prints it, and prints all of the author name values.

In [ ]:
fetchUrl = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
for pubNumber in range(0,limit):
    paramDict = {
        'tool': toolName, 
        'email': emailAddress,
        'db': 'pubmed', 
        'retmode': 'xml', 
        'rettype': 'abstract', 
        'id': uidList[pubNumber]
    }
    response = requests.get(fetchUrl, params=paramDict)
    pubData = response.text  # the response text is XML
    #print(pubData)  # uncomment this line to see the XML

    # process the returned XML, see https://docs.python.org/2/library/xml.etree.elementtree.html
    root = et.fromstring(pubData)
    title = root.findall('.//ArticleTitle')[0].text
    try:
        print('\n' + str(pubNumber) + ' ' + uidList[pubNumber] + ' ' + title)
    except:
        print('\n' + str(pubNumber))
    names = root.findall('.//Author')
    for name in names:
        lastName = name.find('./LastName').text
        foreName = name.find('./ForeName').text
        print(foreName + ' ' + lastName)
    print()
    
    # See https://www.ncbi.nlm.nih.gov/books/NBK25497/ for usage guidelines. 
    # An API key is required for more than 3 requests per second.
    sleep(0.5) # wait half a second before hitting the API again to avoid getting blocked


# API with access key (Flickr)

## Flickr Developer Guide

The developer guide is at https://www.flickr.com/services/developer/.  The most critical part is the ["Things You Oughta Know" page](https://www.flickr.com/services/developer/api/) in the guide. In particular, it states the limits per account.  Those limits are 3600 queries per hour.  That comes down to 1 API call per second.  Making calls at a faster rate will cause Flickr to expire or turn off your API key.  So we need to make sure that we include throttling in our code to make sure that we don't exceed that rate.

Another important feature is to cache data retrieved from the API so that you don't keep asking for the same information over and over.  

If you are going to use the Flickr API in a serious way, you need to carefully read the entire "Things You Oughta Know" page.

## Getting an API key

Flickr requires a free API key in order to be able to use their API. Steps to getting a key:

1\. Sign up for a Flickr account at https://www.flickr.com/

![sign up](https://raw.githubusercontent.com/HeardLibrary/digital-scholarship/master/code/scrape/pylesson/images/sign-up.png)

![sign up dialog](https://raw.githubusercontent.com/HeardLibrary/digital-scholarship/master/code/scrape/pylesson/images/sign-up-dialog.png)

![verify](https://raw.githubusercontent.com/HeardLibrary/digital-scholarship/master/code/scrape/pylesson/images/confirm-screen.png)

2\. Log in.

![log in](https://raw.githubusercontent.com/HeardLibrary/digital-scholarship/master/code/scrape/pylesson/images/sign-in.png)

3\. Go to https://www.flickr.com/services/apps/create/ and click on `Request an API Key`.  

![request API link](https://raw.githubusercontent.com/HeardLibrary/digital-scholarship/master/code/scrape/pylesson/images/request-api-link.png)

A good app description is something like "This will be a Python script to learn how to use the Flickr API.  It will run on my local computer and be used for learning and testing."

![app description](https://raw.githubusercontent.com/HeardLibrary/digital-scholarship/master/code/scrape/pylesson/images/app-description.png)

4\. Save your Key somewhere where you can remember it. The Secret would be needed for write authentication, but we won't be using it.  You can save it in case you need it in the future.

![app key](https://raw.githubusercontent.com/HeardLibrary/digital-scholarship/master/code/scrape/pylesson/images/app-key.png)

Click on the link to take you to the API documentation at https://www.flickr.com/services/api/.  This is where we can see what methods we can use to access data at Flickr.

## Keeping your key safe

It's a really bad idea to hard code your API key in your script.  If you give the script to someone else or post it somewhere like GitHub, you've compromised your account.

The following code loads the key from a text file called `flickr_api_key.txt` in your home directory.  You need to create that file in your home directory using a text editor. The file should contain only the key without a newline character at the end.

Change the code if you want to keep the key file somewhere else. The script uses the home directory because everyone has a home directory and the syntax is platform-independent.  It's not a good idea to keep the key in the same directory as the script. Keeping it in a different directory makes it less likely that you will accidentally give the key to someone along with the script.

In [ ]:
from pathlib import Path
import requests
import json
from time import sleep
import webbrowser

home = str(Path.home()) #gets path to home directory; supposed to work for Win and Mac
keyFilename = 'flickr_api_key.txt'
apiKeyPath = home + '/' + keyFilename

try:
    with open(apiKeyPath, 'rt', encoding='utf-8') as fileObject:
        api_key = fileObject.read()
        print(api_key) # delete this line once the script is working; don't want the key as part of the notebook
except:
    print(keyFilename + ' file not found - is it in your home directory?')

## Generating the parameter string

There are all kinds of "methods" (Flickr's term for search types) for finding information about images on Flickr.  The list of methods is on [this page](https://www.flickr.com/services/api/).  We will use the `flickr.photos.search` method to search for images on a particular topic. The details are on [this page](https://www.flickr.com/services/api/flickr.photos.search.html).

Another very useful type of search is `flickr.people.getPublicPhotos` (described [here](https://www.flickr.com/services/api/flickr.people.getPublicPhotos.html)), which can be used to download metadata about all of the images posted by a particular person.

You should note that the approach taken by Flickr about endpoint URLs is different from some other APIs. Some APIs have a different endpoint URL for each kind of search. Flickr uses the same endpoint URL (`https://www.flickr.com/services/rest`) for all of its services and differentiates its kinds of searches by one of the parameters passed to the API: `method`.

In this example, we'll be restricting our search to photos by Arthur Chapman, an Australian biologist who posts a lot of wildlife photos on Flickr.  We restrict the search by including his `user_id` as one of the parameters.  To remove that restriction, we can delete that key/value pair from the parameter list.

The actual search is restricted by the value given for the `tags` parameter; the script below looks for photos of kangaroos by searching for an exact tag. If a user has not tagged their photos, they will not be found by this search. 

An alternative is to use the `text` parameter.  As the API documentation states "Photos who's title, description or tags contain the text will be returned."  The API will return at most 4000 results for a given search. If the search produces more results than that, you need to make it more specific.

For testing purposes, we've set the query to retrieve only one image.  To do something useful, you'll probably want to increase the value of `per_page`.  The `page` parameter can be used to control paging. 

The `extras` parameter allows you to ask for a wide variety of extra information about the image record.  We will use it to get the access URL of the original photo so that we can download it to our hard drive.

In [ ]:
userId = '32005048@N06' # Arthur Chapman's ID
endpointUrl = 'https://www.flickr.com/services/rest'
paramDict = {
    'method' : 'flickr.photos.search',
    'tags' : 'kangaroo',
    'extras' : 'url_o',
    'per_page' : '1',  # default is 100, maximum is 500. Use paging to retrieve more than 500.
    'page' : '1',
    'user_id' : userId,
    'oauth_consumer_key' : api_key,
    'nojsoncallback' : '1', # this parameter causes the API to return actual JSON instead of its weird default string
    'format' : 'json' # overrides the default XML serialization for the search results
    }

#let $query := 'method=flickr.people.getPhotos&amp;extras=license,media,original_format,description,date_taken,geo,tags,machine_tags,url_t,url_o&amp;per_page=500&amp;page='||$page||'&amp;user_id='||$userID||'&amp;oauth_consumer_key='||$consumerKey

metadataResponse = requests.get(endpointUrl, params = paramDict)

print(metadataResponse.url)
data = metadataResponse.json()
print(json.dumps(data, indent=4))
sleep(1) # reminder that if this is put into a loop, wait a second to avoid getting blocked

The `total` value is an important bit of information that you can see after running the code above. It shows the total number search results that are available.  If you are writing code to process many search results, you can perform the search to return a single result as we did here in order to get the `total` value.  You can then extract that value to help your code decide how to retrieve all of the results (i.e. to set up the number of results per page and how many times a page loop needs to operate in order to retrieve all of the results). 

The two values we are interested in are the image ID (which will allow us to open the images web page) and the `url_o` value (which will allow us to download the image. The paths through the layers of dictionaries and lists in the Python data structure created from the JSON are `pictureId = data['photos']['photo'][0]['id']` and `downloadUrl = data['photos']['photo'][0]['url_o']` respectively.

## Opening the image's web page

The next block of code creates the pictures web page URL, then opens that page in a new browser tab

In [ ]:
pictureId = data['photos']['photo'][0]['id']
pictureUrl = 'https://www.flickr.com/photos/' + userId + '/' + pictureId
print(pictureUrl)
success = webbrowser.open(pictureUrl)

## Download the image

One possible important use of the Flickr API is to acquire copies of the images themselves.  We requested the access URL for the original, full-sized image (`url_o`), but we could have also requested the access URL for a thumbnail or various other sizes of images.  The available sizes are listed [here](https://www.flickr.com/services/api/flickr.photos.getSizes.html). The order of that list corresponds approximately to the keys `url_sq, url_t, url_s, url_q, url_m, url_n, url_z, url_c, url_l, url_o`.

In this GET request, we use the `.content` attribute of the response object (last line of code) to acquire the content sent from the server instead of the `.text` attribute or `.json()` method.  That's because the image is binary (bytes) data that can't be interpreted as either raw text or JSON serialized text. The binary form of the image is also the reason why we use `wb` (write binary, vs. `wt` for text) when we open the output file object.  

We also use the value of the `content-type` key in the response header to decide what kind of file extension to use for the image that we receive.  

The single file is downloaded into the working directory of the script.  You will probably want to change the path of the output file to somewhere else if you want to save a lot of pictures.

This code does not include any throttling since retrieving the images is done directly from an access URL and not the API itself.  

In [ ]:
downloadUrl = data['photos']['photo'][0]['url_o']
print(downloadUrl)

# send a request for the image itself
pictureResponse = requests.get(downloadUrl)

# figure out what the media type is for the image that we retrieved.
pictureMediatype = pictureResponse.headers['content-type']
print(pictureMediatype)
if pictureMediatype == 'image/jpeg':
    pictureExtension = '.jpg'
elif pictureMediatype == 'image/png':
    pictureExtension = '.png'
elif pictureMediatype == 'image/gif':
    pictureExtension = '.gif'
else:
    # stop the script if we don't know the type of image
    sys.exit()

# 'wb' is used to write binary objects
with open(pictureId + pictureExtension, 'wb') as fileObject:
    # .content attribute is used for binary objects instead of .text
    fileObject.write(pictureResponse.content)

## Writing to the Flickr API

The API calls we made were read only. They required us to identify ourselves using an API key, but did not actually require authentication using the secret.  The Flickr API requires authentication for any write operations, such as uploading images or changing image metadata.  Unfortunately, Flickr uses the OAuth1 authentication method which is difficult to use.  In the next section, we will take a look at a writing example that uses the simpler OAuth2 method.

# API with OAuth2 authentication (Twitter)

Every API that allows data upload will require some kind of authentication.  Some APIS, like Twitter, require authentication even if you are only doing read operations.  

The DiSC office has an educational account with Twitter that allows us to give student developer accounts associated with the office for a limited time.  Since it's an educational account, DiSC isn't held responsible if students abuse their accounts.  Nevertheless, we expect that you will abide by Twitter's policies for appropriate API use.  

You should have received an invitation for a Twitter developer account.  Follow the instructions [here](https://heardlibrary.github.io/digital-scholarship/script/python/authenticate/) to authenticate your account.

Once you have your API key and API secret key recorded somewhere safe, return to these instructions.  The sections of the web page from "Understanding API authorization" and beyond are duplicated here with live code cells.

## OAuth2 authentication

The OAuth1 authentication has basically one complex system for granting client access to online resources.  OAuth2 has four different authentication systems of differing complexity depending on the mode of access. (For the clearest introduction to this topic, I recommend [this Medium article](https://medium.com/google-cloud/understanding-oauth2-and-building-a-basic-authorization-server-of-your-own-a-beginners-guide-cf7451a16f66).) 

![Authorization Code OAuth2 flow](https://raw.githubusercontent.com/HeardLibrary/digital-scholarship/master/code/scrape/pylesson/images/oauth2-authorization-code.png)

*Image from "Understanding OAuth2 and Building a Basic Authorization Server of Your Own: A Beginner’s Guide" by Ratros Y. in [medium.com](https://medium.com/google-cloud/understanding-oauth2-and-building-a-basic-authorization-server-of-your-own-a-beginners-guide-cf7451a16f66)*

The more complex "authorization flows" involve a complicated dance between the client, an authorization server, and the server that provides the actual resources that are desired.  These kinds of authorization are used when you do a third-party authorization like "Log in using Facebook" and are necessary when the client is operating as a web page or mobile device app.

![Client Credentials OAuth2 flow](https://raw.githubusercontent.com/HeardLibrary/digital-scholarship/master/code/scrape/pylesson/images/client-credentials.png)

Fortunately, since we are creating a client application on a private computer, we are able to use the least complicated of the OAuth2 flow types: "Client Credentials" (also known as "Application Access" authentication).  The client credentials method sends the client’s identifier (the API key) and secret (the API secret key) directly to the authorization server. This is only feasible in the situation where the user knows that the identifier and secret can safely be stored on the device from which the client is operating. 

The Client Credentials authorization process has two steps:

1. The client sends its key and secret to get an access token from the authorization server.
2. The client uses the access token to request the resource it wants from the resource server.

The client key and secret do not expire unless they are deleted by the client owner or are revoked by the server owner.  The access token may have a shorter lifespan depending on the limits set by the server owner.  In some cases, they may only last for minutes (in the case of banking) or hours (in the case of shopping), but they may also last for an indefinite amount of time.

The point here is that the client should have only infrequent interactions with the authorization server - interacting only often enough to get access tokens as required.  Once the access token is obtained, it should be saved and reused many times to access the resource server.

## Getting an access token using the Requests-OAuthlib library

Coding all of the complicated steps needed to carry out OAuth2 authentication would be nearly impossible for novice Python programmers.  Fortunately, there is a Python library for that!  The `Requests-OAuthlib` library is not part of the famous `requests` module, but it is built upon it and uses it under the hood.  It has a reasonably good [developer guide](https://requests-oauthlib.readthedocs.io/en/latest/), which has [code examples](https://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html#backend-application-flow) for the Client Credentials flow (which it confusingly calls "Backend Application Flow", a third name for the process!).  The following scripts are based in part on their examples.

As with the Flickr example, the client key and secret should be stored in a file (called `twitter_api_keys.txt`) in the user's home directory.  The key should come first followed by a newline (i.e. hard return) with the secret on the second line (it doesn't matter if the second line ends in a newline).  

In [ ]:
from pathlib import Path
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

home = str(Path.home()) #gets path to home directory; supposed to work for Win and Mac
keyFilename = 'twitter_api_keys.txt'
apiKeyPath = home + '/' + keyFilename

try:
    with open(apiKeyPath, 'rt', encoding='utf-8') as fileObject:
        fileStringList = fileObject.read().split('\n')
        clientKey = fileStringList[0]
        clientSecret = fileStringList[1]
        print('key: ', clientKey) # delete this line once the script is working; don't want printout as part of the notebook
        print('secret: ', clientSecret) # delete this line once the script is working; don't want printout as part of the notebook
except:
    print(keyFilename + ' file not found - is it in your home directory?')

The `Requests-OAuthlib` creates a session object based on the client key.  The `.fetch_token()` method of the session uses the key and secret to get the access token using the `requests` module under the hood.  This is the interaction shown on the left side of the diagram above.

In [ ]:
requestTokenUrl = 'https://api.twitter.com/oauth2/token'

clientObject = BackendApplicationClient(client_id=clientKey)
oauth = OAuth2Session(client=clientObject)
accessTokenJson = oauth.fetch_token(token_url=requestTokenUrl, client_id=clientKey, client_secret=clientSecret)
# access token should be cached/saved and used repeatedly rather than making many requests fo a new token
print(accessTokenJson)
print()
accessToken = accessTokenJson.get('access_token') # extract the value of access_token from the JSON
print(accessToken)

This is the end of the first stage of the process (authentication).  The second stage of the process (acquiring resources), represented by the right half of the diagram above, should be a separate script because we do not want to repeat the authentication step each time we access a resource. The way the script is currently written, the access token needs to be copied from the output of the previous script and pasted as the value assigned to `bearerAccessToken` in the script below.  It would be better to save the access token in a file at the end of the script above and then load it from the file at the start of the script below.  

There are a number of ways that authorization can be carried out once an access key is generated.  In the case of Twitter, the key is sent as part of an `Authorization` header sent as part of the GET request.

The script below sends parameters to the API requesting the most recent tweets from a particular user.  Before using the script, you need to enter the Twitter handle of the person whose tweets you want to retrieve. The JSON that is returned from the API is massive, so if you want to acquire information different from what's listed below, you'll need to use one of the methods described in the previous lesson to explore it. https://jsoneditoronline.org/ might be a good option for this one.

The Twitter API differs from the Flickr API in that there is a different endpoint for most of the API functions.  The page describing the `GET statuses/user_timeline` endpoint is [here](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline).   

There are many other options for interacting with the API - the [developer guide](https://developer.twitter.com/en/docs/basics/getting-started) provides links to them.  With authentication, you can not only retrieve data from the API, but also upload data.  For example, the [POST statuses/update](https://developer.twitter.com/en/docs/tweets/post-and-engage/api-reference/post-statuses-update) endpoint allows your app to tweet on your behalf.  It uses HTTP POST rather than GET, which requires some changes in the way the `requests` module is used.

In [ ]:
import requests   # best library to manage HTTP transactions

bearerAccessToken = 'access token from first program goes here'
# GET statuses / user_timeline Returns a collection of the most recent Tweets posted by the indicated by the screen_name or user_id parameters.
resourceUrl = 'https://api.twitter.com/1.1/statuses/user_timeline.json'
paramDict = {'screen_name' : 'put twitter handle here', 'count' : '3', 'exclude_replies' : 'true', 'include_rts' : 'false'}
r = requests.get(resourceUrl, headers={'Authorization' : 'Bearer '+ bearerAccessToken}, params = paramDict)
data = r.json()
for tweet in data:
    print('Tweet: ' + tweet['text'])
    print('On: ' + tweet['created_at'])
    print('By: ' + tweet['user']['name'])
    print()